# Pandas — Data Wrangling (45 Worked Examples, 15 per section)

Sections:
1) **Handling missing data**
2) **GroupBy, aggregation, sorting**
3) **Merging and joining DataFrames**

**Per example format**
- *Markdown*: What the function is about and key usage notes
- *Code cell*: Runnable example
- *Markdown*: Optimization options/tips


In [ ]:
import numpy as np
import pandas as pd
np.__version__, pd.__version__

## Section 1 — Handling missing data (15 Examples)

### Detect missing: `isna` / `isnull`
Boolean mask of missing entries (NaN/NaT/None). `isna` and `isnull` are aliases.

In [ ]:
s = pd.Series([1.0, np.nan, None, 4.0])
print(s.isna())
print(s.isnull().sum())

**Optimization options**
- Use vectorized masks; avoid Python loops for NA checks.

### Detect non-missing: `notna` / `notnull`
Inverse of `isna`; useful for filtering to complete cases.

In [ ]:
df = pd.DataFrame({'A':[1, np.nan, 3], 'B':[4, 5, np.nan]})
print(df[df['A'].notna()])

**Optimization options**
- Filter once and reuse the filtered DataFrame to avoid repeated masking.

### Summary of missing by column
Quick NA audit per column.

In [ ]:
df = pd.DataFrame({'A':[1, np.nan, 3], 'B':[np.nan, 5, np.nan], 'C':[1,2,3]})
print(df.isna().sum().sort_values(ascending=False))

**Optimization options**
- Profile early; drop or fix high-NA columns before heavy transforms.

### Replace sentinels then cast: `replace` → `to_numeric`
Convert placeholders like 'NA', '-' to proper NaN before casting.

In [ ]:
raw = pd.Series(['1','NA','3','-'])
clean = pd.to_numeric(raw.replace({'NA':np.nan,'-':np.nan}), errors='coerce')
print(clean)

**Optimization options**
- Normalize representations once at ingestion to avoid repeated coercions.

### Fill with scalar or mapping: `fillna`
Fill different columns with different defaults via dict mapping.

In [ ]:
df = pd.DataFrame({'qty':[1,np.nan,3], 'city':[np.nan,'SF','NY']})
print(df.fillna({'qty':0, 'city':'Unknown'}))

**Optimization options**
- Prefer mapping dict over multiple passes; it’s a single vectorized op.

### Forward/backward fill: `ffill` / `bfill`
Propagate last/next valid observation along an axis; limit controls run length.

In [ ]:
s = pd.Series([1, np.nan, np.nan, 4, np.nan])
print(s.ffill(limit=1))
print(s.bfill())

**Optimization options**
- Use `limit` to prevent long runs of propagated values that may mislead analysis.

### Interpolate numeric data: `interpolate`
Estimate missing numeric values (linear by default).

In [ ]:
s = pd.Series([0, np.nan, 2, np.nan, 4])
print(s.interpolate())

**Optimization options**
- Pick a method (`linear`, `time`, `spline`) that matches the data’s generating process.

### Time-aware interpolation
If index is DatetimeIndex, `method='time'` respects uneven time gaps.

In [ ]:
idx = pd.date_range('2025-01-01', periods=5, freq='D')
s = pd.Series([0, np.nan, 2, np.nan, 4], index=idx)
print(s.interpolate(method='time'))

**Optimization options**
- Set a proper DatetimeIndex for meaningful time interpolation.

### Drop missing: `dropna` with `how`, `thresh`, `subset`
Remove rows/cols with too many NAs.

In [ ]:
df = pd.DataFrame({'A':[1,np.nan,3],'B':[np.nan,np.nan,9],'C':[1,2,3]})
print(df.dropna(thresh=2))
print(df.dropna(axis=1, how='all'))
print(df.dropna(subset=['B']))

**Optimization options**
- Prefer `thresh` for large tables to keep rows with enough signal.

### Conditional NA set: `mask` / `where`
Set out-of-range values to NA without changing shape.

In [ ]:
s = pd.Series([10, -5, 7, 100])
print(s.mask(~s.between(0, 50)))

**Optimization options**
- Branchless vectorization is faster and clearer than loops for capping/outlier NAing.

### Combine sources: `combine_first`
Fill NA in left with aligned values from right.

In [ ]:
a = pd.Series([1, np.nan, 3], index=list('abc'))
b = pd.Series([9, 8, 7], index=list('bcd'))
print(a.combine_first(b))

**Optimization options**
- Align on index; avoid manual joins + `fillna` when this does the job.

### Groupwise imputation with `transform`
Fill within groups (e.g., per city mean).

In [ ]:
df = pd.DataFrame({'city':['SF','SF','NY','NY'],'score':[1.0,np.nan,2.0,np.nan]})
means = df.groupby('city')['score'].transform('mean')
print(df.assign(score=df['score'].fillna(means)))

**Optimization options**
- Use `transform` (not `apply`) to align results back to original rows efficiently.

### Categorical NAs and `fillna`
Fill NAs in categorical data with an explicit category label.

In [ ]:
s = pd.Series(['low', None, 'high', 'low'], dtype='category')
print(s.cat.add_categories(['Unknown']).fillna('Unknown'))

**Optimization options**
- Add categories before fill; avoids implicit dtype changes.

### `convert_dtypes` and nullable dtypes
Upgrade to pandas’ nullable dtypes (Int64, Boolean, string) that support NA.

In [ ]:
df = pd.DataFrame({'i':[1, None, 3], 'b':[True, None, False], 's':['a', None, 'c']})
print(df.convert_dtypes())

**Optimization options**
- Nullable dtypes avoid object fallbacks; improve ops and memory consistency.

### `infer_objects` (legacy)
Try to infer better dtypes for object columns (older helper).

In [ ]:
df = pd.DataFrame({'col': pd.Series([1,2,3], dtype='object')})
print(df.infer_objects().dtypes)

**Optimization options**
- Prefer `convert_dtypes` in modern pandas; it’s more robust.

## Section 2 — GroupBy, aggregation, sorting (15 Examples)

### Basic `groupby().agg`
Aggregate numeric columns by a categorical key.

In [ ]:
df = pd.DataFrame({'k':['A','A','B','B'],'v':[1,2,3,4]})
print(df.groupby('k')['v'].agg(['mean','sum']))

**Optimization options**
- Combine multiple stats in one pass with `agg` to reduce scans.

### Named aggregation
Readable column-specific aggregations.

In [ ]:
df = pd.DataFrame({'team':['X','X','Y'], 'score':[10,20,30], 'time':[5,6,7]})
print(df.groupby('team').agg(avg_score=('score','mean'), total_time=('time','sum')))

**Optimization options**
- Named aggregation documents intent and avoids MultiIndex columns.

### Multi-key groupby
Group by multiple columns.

In [ ]:
df = pd.DataFrame({'k1':['A','A','B','B'],'k2':[1,1,1,2],'v':[1,2,3,4]})
print(df.groupby(['k1','k2'])['v'].sum())

**Optimization options**
- Ensure both keys have matching/clean dtypes; consider category for low-cardinality keys.

### `transform` vs `agg`
`transform` returns aligned results per row (same size).

In [ ]:
df = pd.DataFrame({'grp':['A','A','B','B'],'v':[1,2,3,4]})
print(df.assign(zscore=lambda d:(d['v']-d.groupby('grp')['v'].transform('mean'))/d.groupby('grp')['v'].transform('std')))

**Optimization options**
- Use `transform` for per-row features; `agg` for reductions.

### Group sizes: `size` and `value_counts`
Efficient counts per group or overall.

In [ ]:
df = pd.DataFrame({'city':['SF','SF','NY','LA','LA','LA']})
print(df.groupby('city').size())
print(df['city'].value_counts())

**Optimization options**
- Prefer `value_counts` for single column counts; it’s optimized.

### Distinct counts: `nunique`
Number of unique values per group.

In [ ]:
df = pd.DataFrame({'city':['SF','SF','NY','NY'],'user':[1,2,1,3]})
print(df.groupby('city')['user'].nunique())

**Optimization options**
- Set `dropna=False` if you need to count NaNs distinctly.

### Categoricals + groupby (`observed=True`)
Speed/clarity improvement with categorical groupers.

In [ ]:
df = pd.DataFrame({'k':pd.Categorical(['A','B','A','C'], categories=list('ABCD')), 'v':[1,2,3,4]})
print(df.groupby('k', observed=True)['v'].sum())

**Optimization options**
- Use `observed=True` to skip unused categories and speed up.

### Cumulative per group: `cumcount`, `cumsum`
Running operations within groups.

In [ ]:
df = pd.DataFrame({'grp':['A','A','A','B','B'],'v':[1,2,3,1,2]})
print(df.assign(rank=df.groupby('grp').cumcount()+1, runsum=df.groupby('grp')['v'].cumsum()))

**Optimization options**
- Cumulative ops avoid expensive `apply` when you only need running totals.

### Top-N within group
Filter top rows per group by a metric.

In [ ]:
df = pd.DataFrame({'grp':list('AAABBB'), 'score':[5,2,7,3,9,1]})
rank = df.groupby('grp')['score'].rank(method='first', ascending=False)
print(df[rank<=2].sort_values(['grp','score'], ascending=[True,False]))

**Optimization options**
- Use `rank` + mask instead of `groupby.apply(lambda g: g.nlargest(k))` for speed.

### Quantiles per group: `quantile`
Compute quantiles for each group.

In [ ]:
df = pd.DataFrame({'g':['A','A','B','B','B'],'v':[1,5,2,3,10]})
print(df.groupby('g')['v'].quantile([0.5, 0.9]))

**Optimization options**
- Prefer vectorized `quantile` over manual sorting/slicing.

### Describe per group
Full descriptive stats by group.

In [ ]:
df = pd.DataFrame({'g':['A','A','B','B'],'v':[1,2,3,4]})
print(df.groupby('g')['v'].describe())

**Optimization options**
- Use `describe()` to sanity-check distributions before modeling.

### Sort values and indices
Sort by multiple keys and control stability.

In [ ]:
df = pd.DataFrame({'a':[2,1,2,1],'b':[3,3,1,2]})
print(df.sort_values(['a','b'], ascending=[True,False], kind='mergesort'))

**Optimization options**
- Use stable sort (`kind='mergesort'`) when you need to preserve tie order.

### Sort within groups (rank)
Assign order within each group then sort.

In [ ]:
df = pd.DataFrame({'g':['A','A','B','B'],'v':[3,1,4,2]})
df['r'] = df.groupby('g')['v'].rank(ascending=False, method='first')
print(df.sort_values(['g','r']))

**Optimization options**
- Ranking is often faster and clearer than `groupby.apply(sorted)`.

### GroupBy on time: `Grouper(freq=...)`
Bucket by time period then aggregate.

In [ ]:
idx = pd.date_range('2025-01-01', periods=6, freq='D')
df = pd.DataFrame({'ts':idx, 'v':[1,2,3,4,5,6]}).set_index('ts')
print(df.groupby(pd.Grouper(freq='3D')).sum())

**Optimization options**
- Use `Grouper` for resampling-like buckets without converting to Series.

### `apply` for custom logic (use sparingly)
Run arbitrary functions per group when needed.

In [ ]:
df = pd.DataFrame({'g':['A','A','B','B'],'v':[1,10,2,3]})
print(df.groupby('g')['v'].apply(lambda s: (s-s.mean())/s.std()))

**Optimization options**
- Prefer built-ins (`agg`, `transform`) first; `apply` is slower and harder to parallelize.

## Section 3 — Merging and joining DataFrames (15 Examples)

### Inner join: `pd.merge` on key
Keep rows where keys match in both frames.

In [ ]:
left = pd.DataFrame({'id':[1,2,3], 'L':['a','b','c']})
right = pd.DataFrame({'id':[2,3,4], 'R':['u','v','w']})
print(pd.merge(left, right, on='id', how='inner'))

**Optimization options**
- Ensure key dtypes match to avoid empty joins or implicit casts.

### Left join with suffixes
Preserve all left rows; disambiguate overlapping column names via `suffixes`.

In [ ]:
l = pd.DataFrame({'id':[1,2,3], 'val':[10,20,30]})
r = pd.DataFrame({'id':[2,3,3], 'val':[200,300,999]})
print(pd.merge(l, r, on='id', how='left', suffixes=('_l','_r')))

**Optimization options**
- Select only required columns before merge to reduce memory/CPU.

### Multi-key join
Join on multiple columns.

In [ ]:
l = pd.DataFrame({'k1':['A','A','B'],'k2':[1,2,1],'v':[10,20,30]})
r = pd.DataFrame({'k1':['A','B','B'],'k2':[2,1,2],'w':[5,6,7]})
print(pd.merge(l, r, on=['k1','k2'], how='outer'))

**Optimization options**
- Sort keys or set as index beforehand for readability; ensure no unintended dupes.

### Join with different key names: `left_on`/`right_on`
When column names differ across frames.

In [ ]:
l = pd.DataFrame({'id_left':[1,2,3], 'v':[10,20,30]})
r = pd.DataFrame({'id_right':[2,3,4], 'w':['x','y','z']})
print(pd.merge(l, r, left_on='id_left', right_on='id_right', how='inner'))

**Optimization options**
- Consider renaming keys ahead of time to simplify downstream code.

### Outer join (full)
Keep all keys from both sides; missing filled with NaN.

In [ ]:
l = pd.DataFrame({'id':[1,2], 'v':[10,20]})
r = pd.DataFrame({'id':[2,3], 'w':[200,300]})
print(pd.merge(l, r, on='id', how='outer').sort_values('id'))

**Optimization options**
- Use outer joins for reconciliation; follow with NA audit/fill.

### Right join
Keep all right rows; match from left.

In [ ]:
l = pd.DataFrame({'id':[1,2,3], 'v':['a','b','c']})
r = pd.DataFrame({'id':[2,3,4], 'w':['u','v','w']})
print(pd.merge(l, r, on='id', how='right'))

**Optimization options**
- Prefer `left` + swapping variables to keep direction consistent across codebases.

### Index-based join: `DataFrame.join`
Join on index (or on key then set_index).

In [ ]:
l = pd.DataFrame({'v':[10,20,30]}, index=[1,2,3])
r = pd.DataFrame({'w':[200,300]}, index=[2,3])
print(l.join(r, how='left'))

**Optimization options**
- Set key as index once, then reuse `join` for multiple lookups.

### Concatenation vs merge: `pd.concat`
Stack vertically (axis=0) or align columns horizontally (axis=1).

In [ ]:
a = pd.DataFrame({'id':[1,2], 'v':['x','y']})
b = pd.DataFrame({'id':[3], 'v':['z']})
print(pd.concat([a,b], ignore_index=True))
print(pd.concat([a.set_index('id'), b.set_index('id')], axis=1))

**Optimization options**
- Use `concat` for union-like stacking; use `merge` for relational joins.

### Time-based asof join: `merge_asof`
Match on nearest key not after the left key (requires sorted keys).

In [ ]:
prices = pd.DataFrame({'time':[1,2,4,7], 'px':[10,11,13,15]})
trades = pd.DataFrame({'time':[2,3,6], 'qty':[5,9,1]})
print(pd.merge_asof(trades.sort_values('time'), prices.sort_values('time'), on='time'))

**Optimization options**
- Always sort by the key; set `tolerance`/`direction` to constrain matches.

### Ordered merge: `merge_ordered`
Merge preserving order; useful for time series with different keys.

In [ ]:
a = pd.DataFrame({'t':[1,4], 'A':[10,40]})
b = pd.DataFrame({'t':[2,3,5], 'B':[20,30,50]})
print(pd.merge_ordered(a, b, on='t'))

**Optimization options**
- After merge, forward-fill (`ffill`) to carry last known values if needed.

### Join validation: `validate`
Detect unexpected join cardinalities (e.g., 1:1, 1:m).

In [ ]:
l = pd.DataFrame({'id':[1,2,3], 'v':[10,20,30]})
r = pd.DataFrame({'id':[2,2,3], 'w':[200,201,300]})
try:
    pd.merge(l, r, on='id', how='left', validate='one_to_one')
except Exception as e:
    print('Validation error:', type(e).__name__)

**Optimization options**
- Always validate for critical joins to catch data quality issues early.

### Update values from another frame: `update`
Align on index/columns and update in place for non-NA values.

In [ ]:
base = pd.DataFrame({'id':[1,2,3],'price':[np.nan, 9.0, 12.0]}).set_index('id')
new  = pd.DataFrame({'id':[1,2],'price':[7.5, 10.0]}).set_index('id')
base.update(new)
print(base)

**Optimization options**
- `update` is simpler than merge+where for small, aligned patches.

### Prevent column collisions: `suffixes` & select
Avoid accidentally overwriting columns with same names.

In [ ]:
l = pd.DataFrame({'id':[1,2,3],'val':[1,2,3]})
r = pd.DataFrame({'id':[2,3],'val':[20,30],'extra':['a','b']})
print(pd.merge(l, r[['id','val']], on='id', how='left', suffixes=('_old','_new')))

**Optimization options**
- Select only needed columns from the right to reduce memory and confusion.

### Efficiency: set indexes before repeated joins
Pre-indexing keys can make repeated joins clearer and sometimes faster.

In [ ]:
l = pd.DataFrame({'id':[1,2,3],'v':[10,20,30]}).set_index('id')
r = pd.DataFrame({'id':[2,3],'w':[200,300]}).set_index('id')
print(l.join(r, how='left'))

**Optimization options**
- For many joins on the same key, keep it as index and reuse `join`.